<hr style="border:2px solid gray"> </hr>

# Machine Learning
<hr style="border:2px solid gray"> </hr>

# Step 0. Import packages and Global Settings

In [1]:
import os, glob, numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers, callbacks, regularizers, optimizers
import matplotlib.pyplot as plt

train_name = 'train.hdf5'
test_name = 'test.hdf5'
category_file_name = 'Categories.txt'
model_dir = './model'

group = 2

# Step 1. Data Loading

In [2]:
import h5py
from tensorflow.keras.utils import HDF5Matrix

X_train, X_test, y_train, y_test = HDF5Matrix(train_name, 'X'), HDF5Matrix(test_name, 'X'), HDF5Matrix(train_name, 'y'), HDF5Matrix(test_name, 'y')

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

Instructions for updating:
Training with HDF5Matrix is not optimized for performance. Instead, we recommend using https://github.com/tensorflow/io to load your HDF5 data into a tf.data Dataset and passing that dataset to Keras.
(42606, 128, 128, 3) (42606, 296)
(14202, 128, 128, 3) (14202, 296)


In [3]:
import codecs
category_file = codecs.open(category_file_name, 'r', 'utf-8')
categories = list(map(str.strip, category_file.readlines()))
classes = len(categories)

# Step 2. Define Model

In [4]:
l2_ratio = 0.01
with tf.device('/device:GPU:0'):
    base_model = tf.keras.applications.vgg19.VGG19(weights= None, include_top=False, input_shape=X_train.shape[1:])
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    predictions = layers.Dense(classes, activation= 'softmax')(x)
    model = models.Model(inputs = base_model.input, outputs = predictions)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)      

# Training

In [6]:
history = model.fit(X_train[:], y_train[:], batch_size=48, epochs=1000, validation_data=(X_test[:], y_test[:]))
print("정확도 : %.4f" % (model.evaluate(test)[1]))

Epoch 1/1000
888/888 [==============================] - 218s 246ms/step - loss: 5.6897 - accuracy: 0.0030 - val_loss: 5.6893 - val_accuracy: 0.0024
Epoch 2/1000
888/888 [==============================] - 188s 212ms/step - loss: 5.6873 - accuracy: 0.0034 - val_loss: 5.6896 - val_accuracy: 0.0020
Epoch 3/1000
888/888 [==============================] - 186s 210ms/step - loss: 5.6866 - accuracy: 0.0034 - val_loss: 5.6901 - val_accuracy: 0.0020
Epoch 4/1000
888/888 [==============================] - 186s 210ms/step - loss: 5.6860 - accuracy: 0.0038 - val_loss: 5.6903 - val_accuracy: 0.0020
Epoch 5/1000
888/888 [==============================] - 185s 209ms/step - loss: 5.6857 - accuracy: 0.0040 - val_loss: 5.6906 - val_accuracy: 0.0020
Epoch 6/1000
888/888 [==============================] - 186s 210ms/step - loss: 5.6855 - accuracy: 0.0035 - val_loss: 5.6908 - val_accuracy: 0.0020
Epoch 7/1000
888/888 [==============================] - 186s 210ms/step - loss: 5.6854 - accuracy: 0.0038 - val_

KeyboardInterrupt: 

# Test

In [ ]:
def plot_history(histories, key='binary_crossentropy'):
    plt.figure(figsize=(16,10))

    for name, history in histories:
        val = plt.plot(history.epoch, history.history['val_'+key],
                    '--', label=name.title()+' Val')
        plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
                label=name.title()+' Train')

    plt.xlabel('Epochs')
    plt.ylabel(key.replace('_',' ').title())
    plt.legend()

    plt.xlim([0,max(history.epoch)])

In [ ]:
plot_history([('', history)], 'accuracy')

In [ ]:
model = Sequential([
        Conv2D(32, (7,7), padding="same", input_shape=X_train.shape[1:], activation='relu', strides=(2, 2)),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        #Dropout(0.25),

        Conv2D(64, (4,4), padding="same", activation='relu'),
        Conv2D(64, (4,4), padding="same", activation='relu'),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        #Dropout(0.25),

        Conv2D(128, (2,2), padding="same", activation='relu'),
        Conv2D(128, (2,2), padding="same", activation='relu'),
        MaxPooling2D(pool_size=(3,3), strides=(2,2)),
        #Dropout(0.25),

        Flatten(),
        #Dense(4096, activation='relu',kernel_regularizer=l2(l2_ratio)),
        Dense(1024, activation='relu',kernel_regularizer=l2(l2_ratio)),
        #Dense(512, activation='relu',kernel_regularizer=l2(l2_ratio)),
        Dense(classes, activation='softmax',kernel_regularizer=l2(l2_ratio))
    ])